# 创建图

In [1]:
import pandas as pd
import copy
import json
# 读取轨迹数据
df_traj = pd.read_csv(f'../traj_station_handle/df_3.csv')

# 读取uli_index_dict.json数据
with open(f'/root/autodl-tmp/data/operator3/extra_file/uli_index_dict.json', 'r', encoding='utf-8') as file:
    uli_index_dict = json.load(file)

In [2]:
df_traj.head()

,Unnamed: 0,user,day,trajlen,traj,timestamps
0,17,a8341153b4c9c,20241007,33,"['3-4-261116184', '3-4-50044933', '3-4-1941465...","[1728266569, 1728266581, 1728266633, 172826666..."
1,40,a834118349e95,20241004,31,"['3-4-261286169', '3-4-178853377', '3-4-178481...","[1728007305, 1728008102, 1728008140, 172800819..."
2,43,a8341183b468d,20241002,38,"['3-4-179270403', '3-4-178706948', '3-4-179032...","[1727861511, 1727861772, 1727861811, 172786182..."
3,62,a8341188577cc,20241007,38,"['3-4-261116184', '3-4-188081203', '3-4-178505...","[1728289522, 1728289573, 1728289634, 172828967..."
4,77,a8341189b76bb,20241005,31,"['3-4-28465715', '3-4-178860034', '3-4-1941307...","[1728104455, 1728104456, 1728104462, 172810448..."


In [3]:
# 定义映射函数
def map_traj_values(traj_list):
    return [uli_index_dict.get(uli, uli) for uli in eval(traj_list)]

# 应用映射函数并创建新列
df_traj['traj_index'] = df_traj['traj'].apply(map_traj_values)

In [4]:
df_traj.head()

,Unnamed: 0,user,day,trajlen,traj,timestamps,traj_index
0,17,a8341153b4c9c,20241007,33,"['3-4-261116184', '3-4-50044933', '3-4-1941465...","[1728266569, 1728266581, 1728266633, 172826666...","[5581, 7587, 5218, 1015, 1009, 3857, 583, 3648..."
1,40,a834118349e95,20241004,31,"['3-4-261286169', '3-4-178853377', '3-4-178481...","[1728007305, 1728008102, 1728008140, 172800819...","[5746, 1885, 958, 2650, 2394, 2393, 6159, 2396..."
2,43,a8341183b468d,20241002,38,"['3-4-179270403', '3-4-178706948', '3-4-179032...","[1727861511, 1727861772, 1727861811, 172786182...","[2695, 1649, 2282, 1647, 1650, 1646, 1652, 228..."
3,62,a8341188577cc,20241007,38,"['3-4-261116184', '3-4-188081203', '3-4-178505...","[1728289522, 1728289573, 1728289634, 172828967...","[5581, 3864, 1015, 581, 105, 533, 408, 1239, 2..."
4,77,a8341189b76bb,20241005,31,"['3-4-28465715', '3-4-178860034', '3-4-1941307...","[1728104455, 1728104456, 1728104462, 172810448...","[6127, 1905, 5125, 1075, 6161, 3771, 1885, 270..."


In [5]:
import networkx as nx
import numpy as np
import os

def build_graph_from_txt(traj_df):
    # 初始化有向图
    G = nx.DiGraph()
    all_nodes = set()

    # 第一步: 收集所有节点
    for index, row in traj_df.iterrows():
        traj_index = row['traj_index']
        for i in range(len(traj_index)):
            u = traj_index[i]
            all_nodes.add(u)

    # 第二步: 将所有节点添加到图中
    for node in all_nodes:
        G.add_node(node)

    # 第三步: 根据文件内容添加边
    for index, row in traj_df.iterrows():
        traj_index = row['traj_index']
        for i in range(len(traj_index) - 1):
            u, v = traj_index[i], traj_index[i + 1]
            # 检查图中是否已经存在从 u 到 v 的边
            if G.has_edge(u, v):
                # 如果边已经存在, 增加边的权重
                G.edges[u, v]['weight'] += 1
            else:
                # 如果边不存在, 添加新边, 权重初始化为 1
                G.add_edge(u, v, weight=1)

    for node in G.nodes:
        G.add_edge(node, node, weight=1)

    nodelist = G.nodes()
    A = nx.adjacency_matrix(G, nodelist=nodelist)
    np.savetxt(f'/root/autodl-tmp/data/operator3/graph/graph_A.csv', A.todense(), delimiter=',')
    return G

In [6]:
build_graph_from_txt(df_traj)

In [7]:
df = pd.read_csv(f'/root/autodl-tmp/data/operator3/graph/graph_A.csv')

# 检查所有值是否都为0
all_zero = (df == 0).all().all()

if all_zero:
    print("CSV文件中的所有值都为0")
else:
    print("CSV文件中存在不为0的值")

CSV文件中存在不为0的值
